In [16]:
%reload_ext autoreload
%autoreload 2

import shutil
import io
import os
from subtle.utils.io import load_file
import matplotlib.pyplot as plt
import numpy as np
import sigpy as sp
from skimage.transform import resize
from scipy.ndimage.morphology import binary_fill_holes
from subtle.data_loaders.slice_loader import SliceLoader
import subtle.subtle_metrics as sumetrics
import subtle.utils.io as suio
from subtle.utils.experiment import get_config, get_experiment_data
import sigpy as sp
from scipy.ndimage.morphology import binary_fill_holes
from train import eval_model
from glob import glob
from subtle.subtle_preprocess import dcm_to_sitk
from tqdm import tqdm_notebook as tqdm
from torchvision.transforms.functional import resize
import pandas as pd
import torch
import SimpleITK as sitk

plt.set_cmap('gray')
plt.rcParams['figure.figsize'] = (10, 8)

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

def process_mpr_sag(vol, plane):
    if plane == 'ax':
        tr = (1, 0, 2)
        x1 = 0
    else:
        tr = (2, 0, 1)
        x1 = 0
    vol = vol.transpose(tr)
    vol = np.rot90(vol, k=3, axes=(1, 2))
    vol = sp.util.resize(vol, [vol.shape[0], vol.shape[1], 240])
    return vol

<Figure size 1000x800 with 0 Axes>

In [ ]:
data = np.load('/home/srivathsa/projects/studies/gad/stanford/preprocess/data/Patient_0085_full.npy')
print(data.shape)

In [ ]:
sl = np.load('/home/srivathsa/projects/studies/gad/stanford/preprocess/slices/Patient_0088/ax/154.npy')[2]
print(sl.min(), sl.max())
vgg_ip = torch.from_numpy(sl[None, None]).to('cuda')
vgg_ip = vgg_ip.repeat(1, 3, 1, 1)
vgg_ip = torch.clip(vgg_ip, min=torch.tensor(0.0).to('cuda'), max=vgg_ip.max())
print(vgg_ip.min(), vgg_ip.max())

vgg_rs = resize(vgg_ip, [224, 224], antialias=False).cpu().numpy()
print(vgg_rs.min(), vgg_rs.max())

plt.imshow(vgg_rs[0, 0])
print(vgg_rs.min(), vgg_rs.max())

In [ ]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import PIL.Image as Image

tf_size_guidance = {
    'compressedHistograms': 0,
    'images': 65,
    'scalars': 0,
    'histograms': 0
}

log_path = '/home/srivathsa/projects/studies/gad/stanford/train/tb/stanford_sri_enh_vgg_1680028723/events.out.tfevents.1680028727.subtle-dgx'
event_acc = EventAccumulator(log_path, size_guidance=tf_size_guidance)
event_acc.Reload()



In [ ]:
def byte_to_array(img_bstr):
    return np.array(Image.open(io.BytesIO(img_bstr)))

In [ ]:
imgs = event_acc.Images('Validation_0')
im_arr = []
for idx, img in enumerate(imgs):
    img_bstr = img.encoded_image_string
    img = byte_to_array(img_bstr)
    im_arr.append(img)
im_arr = np.array(im_arr)
print(im_arr.shape)

In [ ]:
data = suio.load_file('/home/srivathsa/projects/studies/gad/tiantan/preprocess/data/NO118.h5').transpose(1, 0, 2, 3)
pre, low, full = data

print(pre.min(), pre.max())
print(low.min(), low.max())
print(post.min(), post.max())

In [ ]:
data = np.load('/home/srivathsa/projects/studies/gad/stanford/preprocess/slices/Patient_0101/ax/234.npy')
pre, low, post, enh = data

print(pre.min(), pre.max())
print(low.min(), low.max())
print(post.min(), post.max())
print(enh.min(), enh.max())


plt.imshow(post)

In [ ]:
bpath = '/home/srivathsa/projects/studies/gad/stanford/preprocess/slices'
fp_npys = [fp for fp in glob('{}/**/*.npy'.format(bpath), recursive=True)]

nan_cases = []

for fp in tqdm(fp_npys, total=len(fp_npys)):
    d = np.load(fp)
    if d.min() < 0:
        cnum = [p for p in fp.split('/') if 'Patient' in p][0]
        if cnum not in nan_cases:
            nan_cases.append(cnum)

In [ ]:
# fp_sub = sorted([fp for fp in fp_npys if 'Patient_0088' in fp])
# for fp in fp_sub:
#     d = np.load(fp)
#     if np.isnan(d.min()) or np.isnan(d.max()):
#         print(fp)
        
# d = np.load('/home/srivathsa/projects/studies/gad/stanford/preprocess/slices/Patient_0088/ax/002.npy')
# plt.imshow(d[-1])

print(d[-1])

In [ ]:
data = np.load('/home/srivathsa/projects/studies/gad/gen_siemens/preprocess/slices/101_Id_007/sag/078.npy')
pre, low, post, enh = data

print(pre.min(), pre.max())
print(low.min(), low.max())
print(post.min(), post.max())
print(enh.min(), enh.max())

In [ ]:
plt.imshow(data.)

In [ ]:
plt.imshow(sl[0])

In [ ]:
bpath = '/home/srivathsa/projects/studies/gad/stanford/preprocess/data'

for fpath in glob('{}/*full*'.format(bpath)):
    os.rename(fpath, fpath.replace('_full', ''))

In [ ]:
plt.imshow(np.hstack([*data[1, 180]]))

In [ ]:
data = np.load('/mnt/raid/srivathsa/gen_siemens/preprocess/slices/Id0039/ax/140.npy')
data[-1] *= 6

In [ ]:
# plt.imshow(np.hstack([*data]))
plt.imshow(data[2] - data[0])

In [ ]:
state_dict = torch.load(
    '/home/srivathsa/projects/studies/gad/gen_siemens/train/checkpoints/gen_siemens_enh_vgg_1679459081/epoch_184.pth'
)

In [ ]:
state_dict = torch.load(
    '/home/srivathsa/projects/studies/gad/tiantan/train/checkpoints/tiantan_sri_enh_vgg_1679284107/epoch_199.pth'
)
print(state_dict['mse'])
# iter_num = int(state_dict['opt_G']['state'][0]['step'])

In [ ]:
data = suio.load_file('/home/srivathsa/projects/studies/gad/tiantan/preprocess/data/NO1.h5')
print(data.shape)

In [ ]:
plt.imshow(data[160, 1] - data[160, 0])
plt.colorbar()

In [ ]:
sl = np.load('/home/srivathsa/projects/studies/gad/tiantan/preprocess/slices/NO1/sag/160.npy')
plt.imshow(sl[0, 1] - sl[0, 0])

In [ ]:
relu = torch.nn.ReLU(inplace=True)
ip = torch.from_numpy(np.array([[1, -2], [-0.5, 3]]).astype(np.float32))

In [ ]:
op = relu(ip)
print(op, ip)

In [ ]:
data_loader = SliceLoader(
    data_files=[
        '/home/srivathsa/projects/studies/gad/tiantan/preprocess/slices/Brain3H-600437593'
    ],
    slices_per_input=1,
    slice_axis=[2],
    resize=240,
    use_enh_mask=True
)

In [ ]:
class DummyNS:
    def __init__(self):
        self.batch_size = 8
        self.random_seed = 723
args = DummyNS()

In [ ]:
X, Y = data_loader.__getitem__(63)

In [ ]:
plt.imshow(np.hstack([X[0], X[1], Y[0]]))

In [ ]:
pre = X[:7]
low = X[7:]

# r1 = np.hstack([*pre])
# r2 = np.hstack([*low])
# diff = np.clip(r2-r1, 0, (r2-r1).max())
# img = np.vstack([r2-r1])
# plt.imshow(np.hstack([pre[0], low[0], Y[0], Y[1]]))

In [ ]:
plt.imshow(Y[1])

In [ ]:
from subtle.dnn.generators import GeneratorUNet2D

# Xt = torch.from_numpy(X[None].astype(np.float32))
net = GeneratorUNet2D(num_channel_input=14, num_channel_output=1).to('cuda')
# Y_pred = net(Xt)

In [ ]:
sum(dict((p.data_ptr(), p.numel()) for p in net.parameters()).values())

### SSIM Loss implementation

In [ ]:
import math
import torch.nn.functional as F

def extract_image_patches(x, kernel=3, stride=3, dilation=1):
    # Do TF 'SAME' Padding
    b,c,h,w = x.shape
    h2 = math.ceil(h / stride)
    w2 = math.ceil(w / stride)
    pad_row = (h2 - 1) * stride + (kernel - 1) * dilation + 1 - h
    pad_col = (w2 - 1) * stride + (kernel - 1) * dilation + 1 - w
    x = F.pad(x, (pad_row//2, pad_row - pad_row//2, pad_col//2, pad_col - pad_col//2))
    
    # Extract patches
    patches = x.unfold(2, kernel, stride).unfold(3, kernel, stride)
    patches = patches.permute(0,4,5,1,2,3).contiguous()
    
    return patches #patches.view(b,-1,patches.shape[-2], patches.shape[-1])

def ssim_loss(y_true, y_pred, kernel=(3, 3), k1=.01, k2=.03, kernel_size=3, max_value=1.):
    # ssim parameters
    cc1 = (k1 * max_value) ** 2
    cc2 = (k2 * max_value) ** 2

    patches_true = extract_image_patches(y_true)
    patches_pred = extract_image_patches(y_pred)
    
    bs, c1, c2, c3, w, h = patches_pred.shape
    patches_true = torch.reshape(patches_true, [-1, c1*c2*c3, w, h])
    patches_pred = torch.reshape(patches_pred, [-1, c1*c2*c3, w, h])

    # Get mean
    u_true = torch.mean(patches_true, dim=1)
    u_pred = torch.mean(patches_pred, dim=1)

    # Get variance
    var_true = torch.var(patches_true, dim=1, unbiased=False)
    var_pred = torch.var(patches_pred, dim=1, unbiased=False)

    # Get covariance
    covar_true_pred = torch.mean(patches_true * patches_pred, dim=1) - (u_true * u_pred)
    
    # compute ssim and dssim
    ssim = (2 * u_true * u_pred + cc1) * (2 * covar_true_pred + cc2)
    
    denom = (torch.square(u_true) + torch.square(u_pred) + cc1) * (var_pred + var_true + cc2)
    ssim /= denom
    return torch.mean((1.0 - ssim) / 2.0)

In [ ]:
ssim = ssim_loss(ip1, ip2)

In [ ]:
data = suio.load_file('/home/srivathsa/projects/studies/gad/tiantan/preprocess/data/Case1.h5')

pre = data[:, 0]
low = data[:, 1]

ip1 = torch.from_numpy(pre[98, 8:-8, 8:-8][None, None].astype(np.float32))
ip2 = torch.from_numpy(low[98, 8:-8, 8:-8][None, None].astype(np.float32))

In [ ]:
print(ssim.numpy())

### VGG-19 Perceptual Loss implementation

In [ ]:
from subtle.subtle_loss import VGGLoss

vgg_loss = VGGLoss(fpath_ckp='/home/srivathsa/projects/vgg19_imagenet_from_tf.pth', img_resize=256).to('cuda')

In [ ]:
data = np.load('/home/srivathsa/projects/studies/gad/stanford/preprocess/slices/Patient_0088/sag/150.npy')
data2 = np.load('/home/srivathsa/projects/studies/gad/stanford/preprocess/slices/Patient_0088/sag/155.npy')

pre = data[0]
pre2 = data2[0]

low = data[1]
low2 = data2[1]

full = data[2]
full2 = data2[2]

ip1 = torch.from_numpy(pre[None, None].astype(np.float32)).to('cuda')
ip2 = torch.from_numpy(low[None, None].astype(np.float32)).to('cuda')
ip3 = torch.from_numpy(full[None, None].astype(np.float32)).to('cuda')
ip4 = torch.from_numpy(full2[None, None].astype(np.float32)).to('cuda')

In [ ]:
lval = vgg_loss(ip3, ip4)
print(lval)

In [ ]:
plt.imshow(np.hstack([full, full2]))

In [ ]:
from PIL import Image
import sigpy as sp

imgnet1 = np.array(Image.open('/home/srivathsa/projects/studies/gad/imagenet1.jpeg'))
imgnet1 = sp.util.resize(imgnet1, (224, 224, 3))

imgnet2 = np.array(Image.open('/home/srivathsa/projects/studies/gad/imagenet2.jpeg'))
imgnet2 = sp.util.resize(imgnet2, (224, 224, 3))

print(imgnet1.min(), imgnet1.max())

### General Siemens Model

In [ ]:
import subtle.utils.io as suio

data = suio.load_file('/home/srivathsa/projects/studies/gad/gen_siemens/preprocess/data/Prisma14.h5')
print(data.shape)

### Check intensities

In [ ]:
src_path = '/home/srivathsa/projects/studies/gad/tiantan/preprocess/slices'
cases = sorted([c.split('/')[-1] for c in glob('{}/*'.format(src_path))])

range_info = []

planes = ['ax', 'sag', 'cor']

for cnum in tqdm(cases, total=len(cases)):
    for plane in planes:
        fpath_nps = sorted([fp for fp in glob('{}/{}/{}/*.npy'.format(src_path, cnum, plane))])
        pre, low, full, mask = np.array([np.load(d) for d in fpath_nps]).transpose(1, 0, 2, 3)
#         sl = pre.shape[0] // 2

#         row1 = np.hstack([pre[sl], low[sl], full[sl]])
#         row2 = np.hstack([low[sl]-pre[sl], full[sl]-pre[sl], mask[sl]])
#         img = np.vstack([row1, row2])
#         fig, (ax1, ax2) = plt.subplots(2)

#         ax1.imshow(row1)
#         ax1.axis('off')

#         ax2.imshow(row2)
#         ax2.axis('off')

#         plt.title(cnum)
#         plt.tight_layout(pad=0, w_pad=0, h_pad=0)
#         plt.margins(y=0)
#         plt.savefig('{}/plots/{}_{}.png'.format(src_path, cnum, plane), dpi=200, bbox_inches='tight')
#         plt.clf()
#         plt.close()
        
        range_info.append({
            'case': cnum,
            'plane': plane,
            'pre_min': pre.min(), 'pre_max': pre.max(), 'pre_mean': pre.mean(),
            'low_min': low.min(), 'low_max': low.max(), 'low_mean': low.mean(),
            'full_min': full.min(), 'full_max': full.max(), 'full_mean': full.mean(),
            'mask_min': mask.min(), 'mask_max': mask.max(), 'mask_mean': mask.mean(),
        })

In [ ]:
df_range = pd.DataFrame(range_info)
df_range.to_csv('/home/srivathsa/projects/studies/gad/tiantan/preprocess/pytorch_port_range_info_04072023.csv')

In [ ]:
train_cases = [
    "Brain2H-600441599", "Brain4H-601044594", "NO1", "NO2", "NO3", "NO4", "NO5", "NO7", "NO8", "NO9", "NO10", 
    "NO11", "NO12", "NO13", "NO15", "NO16", "NO17", "NO19", "NO20", "NO21", "NO22", "NO23", "NO24"
]

In [ ]:
df_range[[
    'case', 'pre_max', 'low_max', 'full_max', 'mask_max', 'plane'
]].query('plane == "cor" and case in @train_cases')

In [ ]:
data = np.load('/home/srivathsa/projects/studies/gad/stanford/preprocess/data/Patient_0126.npy')

In [ ]:
print(data.shape)
pre, low, full = data[1].transpose(1, 0, 2, 3)

plt.imshow(low[:, 256, :])

### Inference test and DICOM generation

In [2]:
data_loader = SliceLoader(
    data_files=[
        '/home/srivathsa/projects/studies/gad/stanford/preprocess/slices/Patient_0101'
    ],
    slices_per_input=7,
    slice_axis=[0],
    resize=512,
    use_enh_mask=True,
    shuffle=False
)

In [3]:
from subtle.dnn.generators import GeneratorUNet2D
net = GeneratorUNet2D(num_channel_input=14, num_channel_output=1).to('cuda')

In [4]:
state_dict = torch.load(
    '/home/srivathsa/projects/studies/gad/stanford/train/checkpoints/stanford_sri_enh_vgg_1681099486/best_mse.pth',
    map_location='cpu'
)

In [5]:
net.load_state_dict(state_dict['G'])

<All keys matched successfully>

In [6]:
num_slices = data_loader.__len__()
Y_pred = []
for idx in tqdm(np.arange(num_slices), total=num_slices):
    X, Y = data_loader.__getitem__(idx)
    Xinp = torch.from_numpy(X[None].astype(np.float32)).to('cuda')
    Yp = net(Xinp)
    Yp = Yp.detach().cpu().numpy()
    
    Y_pred.append(Yp[0, 0])
Y_pred = np.array(Y_pred)

/home/srivathsa/miniconda3/envs/gad_torch/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/336 [00:00<?, ?it/s]

In [7]:
ip_dcm = '/home/srivathsa/projects/studies/gad/stanford/data/Patient_0101/13_Ax_BRAVO+C'
ip_sitk = dcm_to_sitk(ip_dcm)
ref_arr = sitk.GetArrayFromImage(ip_sitk)

sc = ref_arr.max() / Y_pred.max()

prefix = 'SubtleGAD pyt: '
desc = 'enh_vgg_v2'
suffix = ''

Y_pred = np.clip(Y_pred, 0, Y_pred.max())
Y_pred = Y_pred * sc
suio.write_dicoms(
    ip_dcm, Y_pred, '/home/srivathsa/projects/studies/gad/stanford/data/pyt_test/Patient_0101_pyt_{}'.format(desc),
    series_desc_pre=prefix, desc=desc, series_desc_post=suffix
)

ImageSeriesReader (0x558b62e20920): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.000144133

/home/srivathsa/miniconda3/envs/gad_torch/lib/python3.7/site-packages/pydicom/valuerep.py:290: UserWarning: The value length (19) exceeds the maximum length of 16 allowed for VR DS. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)
/home/srivathsa/miniconda3/envs/gad_torch/lib/python3.7/site-packages/pydicom/valuerep.py:290: UserWarning: The value length (18) exceeds the maximum length of 16 allowed for VR DS. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warnings.warn(msg)
/home/srivathsa/miniconda3/envs/gad_torch/lib/python3.7/site-packages/pydicom/valuerep.py:290: UserWarning: The value length (17) exceeds the maximum length of 16 allowed for VR DS. Please see <https://dicom.nema.org/medi

In [ ]:
print(Y_pred.min(), Y_pred.max())

In [ ]:
df_range = pd.read_csv(
    '/home/srivathsa/projects/studies/gad/stanford/preprocess/pytorch_port_range_info_04052023.csv'
)

In [ ]:
df_range

In [ ]:
df_range[[
    'case', 'mask_min', 'mask_max', 'plane'
]].query('plane == "ax"')

In [17]:
dpath_exp = '/home/srivathsa/projects/SubtleGad/train/configs/experiments'
cases = get_experiment_data('tiantan_sri', dataset='train', dirpath_exp=dpath_exp)
cases = cases + get_experiment_data('tiantan_sri', dataset='val', dirpath_exp=dpath_exp)
src_path = '/home/srivathsa/projects/studies/gad/tiantan/preprocess/slices'
dest_path = '/home/srivathsa/projects/studies/gad/tiantan/preprocess/slices/accio_train'

for cnum in cases:
    shutil.copytree('{}/{}'.format(src_path, cnum), '{}/{}'.format(dest_path, cnum))